##  ewf-ext-03-05-01 - Tropical Forest Disturbance Analysis with Sentinel-1 SLC Data

Algorithm to detect SAR amplitude changes (b0) related to landslide events from pre- and post-earthquake Sentinel-1 IW-SLC acquisitions (slave, master).

### Quick link

* [Objective](#objective)
* [Test Site](#test-site)
* [Context](#context)
* [Applicability](#applicability)
* [Data](#data)
* [Service Definition](#service)
* [Parameter Definition](#parameter)
* [Runtime Parameter Definition](#runtime)
* [Workflow](#workflow)
* [Strengths and Limitations](#strengths-limitations) 
* [License](#license)

### <a name="objective">Objective 




### <a name="data">Data

SENTINEL data products are made available systematically and free of charge to all data users including the general public, scientific and commercial users. Radar data will be delivered within an hour of reception for Near Real-Time (NRT) emergency response, within three hours for NRT priority areas and within 24 hours for systematically archived data.

All data products are distributed in the SENTINEL Standard Archive Format for Europe (SAFE) format.

Data products are available in single polarisation (VV or HH) for Wave mode and dual polarisation (VV+VH or HH+HV) and single polarisation (HH or VV) for SM, IW and EW modes.

### <a name="service">Service definition

In [1]:
service = dict([('title', 'ewf-ext-03-05-01 - Tropical Forest Disturbance Analysis with Sentinel-1 SLC Data'),
                ('abstract', 'ewf-ext-03-05-01 - Tropical Forest Disturbance Analysis with Sentinel-1 SLC Data'),
                ('id', 'ewf-ext-03-03-02')])

### Parameter Definition 

Output file format:

* BEAM-DIMAP
* GeoTIFF-BigTIFF

In [2]:
format = dict([('id', 'format'),
               ('value', 'GeoTIFF-BigTIFF'),
               ('title', 'Output file format'),
               ('abstract', 'Output file format: BEAM-DIMAP or GeoTIFF-BigTIFF')])

In [3]:
master = dict([('id', 'master'),
            ('value', 'True'),
            ('title', 'master'),
            ('abstract', 'master')])

In [4]:
slave = dict([('id', 'slave'),
            ('value', 'True'),
            ('title', 'slave'),
            ('abstract', 'slave')])

In [5]:
regionOfInterest = dict([('id', 'regionOfInterest'),
                         ('value', 'POLYGON((-55.1310 -3.7144, -55.1310 -3.2571, -54.5521 -3.2571, -54.5521 -3.7144))'),
                         ('title', 'WKT Polygon for the Region of Interest (-1 if no crop)'),
                         ('abstract', 'Set the value of WKT Polygon')])

### <a name="runtime">Runtime parameter definition
    
The variable values in this section are only relevant for the basic test case. In an actual processing context, the values are replaced by those of the parameters for the process execution.

**Input identifiers**

This is the Sentinel-1 stack of master and slave products' identifiers:

In [6]:
input_identifiers = ('S1A_IW_SLC__1SDV_20170113T092230_20170113T092257_014811_018214_DB60', 'S1A_IW_SLC__1SDV_20170101T092231_20170101T092258_014636_017CCA_B6F3')

**Input references**

This is the Sentinel-1 stack of catalogue references:

In [7]:
input_references = ('https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20170113T092230_20170113T092257_014811_018214_DB60','https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20170101T092231_20170101T092258_014636_017CCA_B6F3')

**Data path**

This path defines where the data is staged-in:

In [8]:
data_path = "/workspace/mall/data"

In [9]:
output_path = ""

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [10]:
from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap

import dateutil.parser as parser
import gc
from datetime import datetime

import cioppy

import gdal
import osr

from shapely.wkt import loads
from shapely.geometry import box

import lxml.etree as etree
import numpy as np
import os

import shutil

import gdal

In [11]:
import lxml.etree as etree
import subprocess
import tempfile
import time
import psutil
from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap
import os

class GraphProcessor():
    
    def __init__(self):
        
        self.root = etree.Element('graph')
    
        version = etree.SubElement(self.root, 'version')
        version.text = '1.0'
        self.pid = None
        self.p = None
   
    def view_graph(self):
        
        print etree.tostring(self.root , pretty_print=True)
        
    def add_node(self, node_id, operator, parameters, source):
    
        xpath_expr = '/graph/node[@id="%s"]' % node_id

        if len(self.root.xpath(xpath_expr)) != 0:

            node_elem = self.root.xpath(xpath_expr)[0]
            operator_elem = self.root.xpath(xpath_expr + '/operator')[0]
            sources_elem = self.root.xpath(xpath_expr + '/sources')[0]
            parameters_elem = self.root.xpath(xpath_expr + '/parameters')

            for key, value in parameters.iteritems():
                p_elem = self.root.xpath(xpath_expr + '/parameters/%s' % key)[0]
                p_elem.text = value
        else:

            node_elem = etree.SubElement(self.root, 'node')
            operator_elem = etree.SubElement(node_elem, 'operator')
            sources_elem = etree.SubElement(node_elem, 'sources')

            if isinstance(source, list):

                for index, s in enumerate(source):
                    if index == 0:  
                        source_product_elem = etree.SubElement(sources_elem, 'sourceProduct')

                    else: 
                        source_product_elem = etree.SubElement(sources_elem, 'sourceProduct.%s' % str(index))

                    source_product_elem.attrib['refid'] = s

            elif source != '':
                source_product_elem = etree.SubElement(sources_elem, 'sourceProduct')
                source_product_elem.attrib['refid'] = source

            parameters_elem = etree.SubElement(node_elem, 'parameters')
            parameters_elem.attrib['class'] = 'com.bc.ceres.binding.dom.XppDomElement'

            for key, value in parameters.iteritems():

                # here I have to adapt the code
                
                if operator == 'BandMaths':
                    
                    if isinstance(value, dict):
                        
                        parameter_elem = etree.SubElement(parameters_elem, key)
                        
                        for key2, value2 in value.iteritems():
                            parameter_elem2 = etree.SubElement(parameter_elem, key2)
                            #parameter_elem.text = value
                            if isinstance(value2, dict):
                                for key3, value3 in value2.iteritems():
                                    parameter_elem3 = etree.SubElement(parameter_elem2, key3)
                                    parameter_elem3.text = value3
                            pass
                    
                    else:
                        parameter_elem = etree.SubElement(parameters_elem, key)
                        parameter_elem.text = value
                else:
                    parameter_elem = etree.SubElement(parameters_elem, key)
                    parameter_elem.text = value
                    
                    
        
        node_elem.attrib['id'] = node_id

        operator_elem.text = operator 

    def save_graph(self, filename):
        
        with open(filename, 'wb') as file:
            file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
            file.write(etree.tostring(self.root, pretty_print=True))
     
    def plot_graph(self):
        
        for node_id in self.root.xpath('/graph/node/@id'):
            

            xpath_expr = '/graph/node[@id="%s"]' % node_id
            
            if len(self.root.xpath(xpath_expr + '/sources/sourceProduct')) != 0:
                print(self.root.xpath(xpath_expr + '/sources/sourceProduct'))[0].attrib['refid']
                print node_id
            else:
                print node_id
        return True
    
    def run(self):
        
        fd, path = tempfile.mkstemp()
        
        try:
        
            self.save_graph(filename=path)
            
            #options = ['/opt/snap6/bin/gpt',
               #'-x',
               #'-c',
               #'2048M',
               #path]
            
            options = ['/workspace/mall/temp/snap6/bin/gpt',
               '-x',
               '-c',
               '2048M',
               path]

            p = subprocess.Popen(options,
                stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE)

            print p.pid
            res, err = p.communicate()
            print res, err
            if p.returncode != 0:
                raise Exception('An error occurred during the execution of gpt (see log)')
            
        except Exception as e:
            with open('stdout.txt', 'wb') as file:
                file.write(res)
                file.close()
            with open('stderr.txt', 'wb') as file:
                file.write(err)
                file.close()
            
            raise
        finally:
            os.remove(path)
        
def get_snap_parameters(operator):
    
    op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

    op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

    return op_params

#### Auxiliary methods

In [12]:
def crop_image(input_image, polygon_wkt, output_path):
    
    dataset = gdal.Open(input_image)

    polygon_ogr = ogr.CreateGeometryFromWkt(polygon_wkt)
    envelope = polygon_ogr.GetEnvelope()
    bounds = [envelope[0], envelope[3], envelope[1], envelope[2]]         
    print bounds
    no_data = dataset.GetRasterBand(1).GetNoDataValue()
    gdal.Translate(output_path, dataset, outputType=gdal.GDT_Float32, projWin=bounds, projWinSRS='EPSG:4326', noData=no_data)

    dataset = None

#### 0. Get S-1 metadata

In [13]:
ciop = cioppy.Cioppy()

search = ciop.search(end_point=input_references[0],
                     params=[],
                     output_fields='enclosure,identifier,startdate,enddate,wkt,orbitNumber,orbitDirection,swathIdentifier,wrsLongitudeGrid',
                     model='EOP') 

search2 = ciop.search(end_point=input_references[1],
                     params=[],
                     output_fields='enclosure,identifier,startdate,enddate,wkt,orbitNumber,orbitDirection,swathIdentifier,wrsLongitudeGrid',
                     model='EOP')

# slave image - always the post-event
input_identifiers_slv_mst = []

if (search[0]['startdate'] < search2[0]['startdate']):
    
    start_date = search[0]['startdate']
    
    input_identifiers_slv_mst.append(input_identifiers[1])
    input_identifiers_slv_mst.append(input_identifiers[0])
    
    mst_date_str = parser.parse(search[0]['startdate']).strftime('%d%b%Y')
    slv_date_str = parser.parse(search2[0]['startdate']).strftime('%d%b%Y')
      
else:
    
    start_date = search2[0]['startdate']
    
    input_identifiers_slv_mst.append(input_identifiers[0])
    input_identifiers_slv_mst.append(input_identifiers[1])
    
    slv_date_str = parser.parse(search[0]['startdate']).strftime('%d%b%Y')
    mst_date_str = parser.parse(search2[0]['startdate']).strftime('%d%b%Y')

if (search[0]['enddate'] > search2[0]['enddate']):
    end_date = search[0]['enddate']
else:
    end_date = search2[0]['enddate']

#### 1. Slave image pre-processing

In [14]:
temp_folder = 'temp'

if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)

In [15]:
mygraph = GraphProcessor()

##### 1.1. Read s-1 product (slave)

In [16]:
operator = 'Read'

index = 0
identifier = input_identifiers_slv_mst[index]
    
parameters = dict()

for param in get_snap_parameters(operator):
    
    if param.getName() == 'file':
        parameters[param.getName()] = os.path.join(data_path, identifier, identifier + '.SAFE', 'manifest.safe')    
    else:
        parameters[param.getName()] = param.getDefaultValue()
        
node_id = 'Read(%s)' % index
    
read_node = node_id
    
print(parameters)
    
mygraph.add_node(node_id, 'Read', parameters, '')

{'formatName': None, 'file': '/workspace/mall/data/S1A_IW_SLC__1SDV_20170113T092230_20170113T092257_014811_018214_DB60/S1A_IW_SLC__1SDV_20170113T092230_20170113T092257_014811_018214_DB60.SAFE/manifest.safe'}


In [17]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read(0)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/mall/data/S1A_IW_SLC__1SDV_20170113T092230_20170113T092257_014811_018214_DB60/S1A_IW_SLC__1SDV_20170113T092230_20170113T092257_014811_018214_DB60.SAFE/manifest.safe</file>
    </parameters>
  </node>
</graph>



##### 1.2. Apply orbit file

In [18]:
operator = 'Apply-Orbit-File'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()


('orbitType', 'Sentinel Precise (Auto Download)')
('polyDegree', '3')
('continueOnFail', 'false')


In [19]:
index = 0
source_node = read_node
    
node_id = 'Apply-Orbit-File(%s)' % index
    
orbit_node = node_id
    
mygraph.add_node(node_id, 'Apply-Orbit-File', parameters, source_node)


In [20]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read(0)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/mall/data/S1A_IW_SLC__1SDV_20170113T092230_20170113T092257_014811_018214_DB60/S1A_IW_SLC__1SDV_20170113T092230_20170113T092257_014811_018214_DB60.SAFE/manifest.safe</file>
    </parameters>
  </node>
  <node id="Apply-Orbit-File(0)">
    <operator>Apply-Orbit-File</operator>
    <sources>
      <sourceProduct refid="Read(0)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <polyDegree>3</polyDegree>
      <orbitType>Sentinel Precise (Auto Download)</orbitType>
      <continueOnFail>false</continueOnFail>
    </parameters>
  </node>
</graph>



##### 1.3. TOPSAR split

In [21]:
operator = 'TOPSAR-Split'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()

('subswath', None)
('selectedPolarisations', None)
('firstBurstIndex', '1')
('lastBurstIndex', '9999')
('wktAoi', None)


In [22]:
slave_split_nodes = []

index_node = 0
source_node = orbit_node

for index, subswath in enumerate(['IW1', 'IW2', 'IW3']):  
    
    parameters['subswath'] =  subswath
    parameters['selectedPolarisations'] = 'VV'

    node_id = 'TOPSAR-Split(%s)' % str(index)
    
    
    slave_split_nodes.append(node_id)

    
    mygraph.add_node(node_id, operator, parameters, source_node)

In [23]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read(0)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/mall/data/S1A_IW_SLC__1SDV_20170113T092230_20170113T092257_014811_018214_DB60/S1A_IW_SLC__1SDV_20170113T092230_20170113T092257_014811_018214_DB60.SAFE/manifest.safe</file>
    </parameters>
  </node>
  <node id="Apply-Orbit-File(0)">
    <operator>Apply-Orbit-File</operator>
    <sources>
      <sourceProduct refid="Read(0)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <polyDegree>3</polyDegree>
      <orbitType>Sentinel Precise (Auto Download)</orbitType>
      <continueOnFail>false</continueOnFail>
    </parameters>
  </node>
  <node id="TOPSAR-Split(0)">
    <operator>TOPSAR-Split</operator>
    <sources>
      <sourceProduct refid="Apply-Orbit-File(0)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
     

##### 1.4. Write

In [24]:
operator = 'Write'

params_list_slave = []

for index, slave_split_node in enumerate(slave_split_nodes):
    parameters = dict()
    
    output_name_slave = os.path.join(temp_folder, 'temp_slave_%s' % index)
    
    node_id = 'Write(%s)' % index
    
    source_node = slave_split_node

    for param in get_snap_parameters(operator):

        if param.getName() == 'file':

            param_value = output_name_slave

        elif param.getName() == 'formatName':

            param_value = 'BEAM-DIMAP'

        else:

            param_value = param.getDefaultValue()


        print (param.getName(), param_value)

        parameters[param.getName()] = param_value
        

        print(node_id)

        print(source_node)
        
    params_list_slave.append(parameters)

('file', 'temp/temp_slave_0')
Write(0)
TOPSAR-Split(0)
('formatName', 'BEAM-DIMAP')
Write(0)
TOPSAR-Split(0)
('deleteOutputOnFailure', 'true')
Write(0)
TOPSAR-Split(0)
('writeEntireTileRows', 'true')
Write(0)
TOPSAR-Split(0)
('clearCacheAfterRowWrite', 'false')
Write(0)
TOPSAR-Split(0)
('file', 'temp/temp_slave_1')
Write(1)
TOPSAR-Split(1)
('formatName', 'BEAM-DIMAP')
Write(1)
TOPSAR-Split(1)
('deleteOutputOnFailure', 'true')
Write(1)
TOPSAR-Split(1)
('writeEntireTileRows', 'true')
Write(1)
TOPSAR-Split(1)
('clearCacheAfterRowWrite', 'false')
Write(1)
TOPSAR-Split(1)
('file', 'temp/temp_slave_2')
Write(2)
TOPSAR-Split(2)
('formatName', 'BEAM-DIMAP')
Write(2)
TOPSAR-Split(2)
('deleteOutputOnFailure', 'true')
Write(2)
TOPSAR-Split(2)
('writeEntireTileRows', 'true')
Write(2)
TOPSAR-Split(2)
('clearCacheAfterRowWrite', 'false')
Write(2)
TOPSAR-Split(2)


In [25]:
slave_tr_nodes = []

for index, slave_split_node in enumerate(slave_split_nodes):
    
    node_id = 'Write(%s)' % index
    
    source_node = slave_split_node
    
    mygraph.add_node(node_id, operator, params_list_slave[index], source_node)
    
    slave_tr_nodes.append(node_id)
      
    print(node_id)
    
    print(source_node)

Write(0)
TOPSAR-Split(0)
Write(1)
TOPSAR-Split(1)
Write(2)
TOPSAR-Split(2)


In [26]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read(0)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/mall/data/S1A_IW_SLC__1SDV_20170113T092230_20170113T092257_014811_018214_DB60/S1A_IW_SLC__1SDV_20170113T092230_20170113T092257_014811_018214_DB60.SAFE/manifest.safe</file>
    </parameters>
  </node>
  <node id="Apply-Orbit-File(0)">
    <operator>Apply-Orbit-File</operator>
    <sources>
      <sourceProduct refid="Read(0)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <polyDegree>3</polyDegree>
      <orbitType>Sentinel Precise (Auto Download)</orbitType>
      <continueOnFail>false</continueOnFail>
    </parameters>
  </node>
  <node id="TOPSAR-Split(0)">
    <operator>TOPSAR-Split</operator>
    <sources>
      <sourceProduct refid="Apply-Orbit-File(0)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
     

##### 1.10. Run

In [27]:
mygraph.run()

1775
Executing processing graph
....10%....20%....30%....40%....50%....60%....70%....80%....90% done.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
SEVERE: org.esa.s2tbx.dataio.gdal.activator.GDALDistributionInstaller: The environment variable LD_LIBRARY_PATH is not set. It must contain the current folder '.'.
INFO: org.hsqldb.persist.Logger: dataFileCache open start



#### 2. Master pre-processing

In [28]:
mygraph = GraphProcessor()

##### 2.1. Read s-1 product (master)

In [29]:
operator = 'Read'

index = 1
identifier = input_identifiers_slv_mst[index]
    
parameters = dict()

for param in get_snap_parameters(operator):
    
    if param.getName() == 'file':
        parameters[param.getName()] = os.path.join(data_path, identifier, identifier + '.SAFE', 'manifest.safe')    
    else:
        parameters[param.getName()] = param.getDefaultValue()
        
node_id = 'Read(%s)' % index
    
read_node = node_id
    
print(parameters)
    
mygraph.add_node(node_id, 'Read', parameters, '')

{'formatName': None, 'file': '/workspace/mall/data/S1A_IW_SLC__1SDV_20170101T092231_20170101T092258_014636_017CCA_B6F3/S1A_IW_SLC__1SDV_20170101T092231_20170101T092258_014636_017CCA_B6F3.SAFE/manifest.safe'}


In [30]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read(1)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/mall/data/S1A_IW_SLC__1SDV_20170101T092231_20170101T092258_014636_017CCA_B6F3/S1A_IW_SLC__1SDV_20170101T092231_20170101T092258_014636_017CCA_B6F3.SAFE/manifest.safe</file>
    </parameters>
  </node>
</graph>



##### 2.2. Apply orbit file

In [31]:
operator = 'Apply-Orbit-File'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()

('orbitType', 'Sentinel Precise (Auto Download)')
('polyDegree', '3')
('continueOnFail', 'false')


In [32]:
index = 1
source_node = read_node
    
node_id = 'Apply-Orbit-File(%s)' % index
    
orbit_node = node_id
    
mygraph.add_node(node_id, 'Apply-Orbit-File', parameters, source_node)

In [33]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read(1)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/mall/data/S1A_IW_SLC__1SDV_20170101T092231_20170101T092258_014636_017CCA_B6F3/S1A_IW_SLC__1SDV_20170101T092231_20170101T092258_014636_017CCA_B6F3.SAFE/manifest.safe</file>
    </parameters>
  </node>
  <node id="Apply-Orbit-File(1)">
    <operator>Apply-Orbit-File</operator>
    <sources>
      <sourceProduct refid="Read(1)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <polyDegree>3</polyDegree>
      <orbitType>Sentinel Precise (Auto Download)</orbitType>
      <continueOnFail>false</continueOnFail>
    </parameters>
  </node>
</graph>



##### 2.3. TOPSAR split

In [34]:
operator = 'TOPSAR-Split'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()

('subswath', None)
('selectedPolarisations', None)
('firstBurstIndex', '1')
('lastBurstIndex', '9999')
('wktAoi', None)


In [35]:
slave_split_nodes = []

index_node = 1
source_node = orbit_node

for index, subswath in enumerate(['IW1', 'IW2', 'IW3']):  
    
    parameters['subswath'] =  subswath
    parameters['selectedPolarisations'] = 'VV'

    node_id = 'TOPSAR-Split(%s)' % str(index)
    
    
    slave_split_nodes.append(node_id)

    
    mygraph.add_node(node_id, operator, parameters, source_node)

In [36]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read(1)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/mall/data/S1A_IW_SLC__1SDV_20170101T092231_20170101T092258_014636_017CCA_B6F3/S1A_IW_SLC__1SDV_20170101T092231_20170101T092258_014636_017CCA_B6F3.SAFE/manifest.safe</file>
    </parameters>
  </node>
  <node id="Apply-Orbit-File(1)">
    <operator>Apply-Orbit-File</operator>
    <sources>
      <sourceProduct refid="Read(1)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <polyDegree>3</polyDegree>
      <orbitType>Sentinel Precise (Auto Download)</orbitType>
      <continueOnFail>false</continueOnFail>
    </parameters>
  </node>
  <node id="TOPSAR-Split(0)">
    <operator>TOPSAR-Split</operator>
    <sources>
      <sourceProduct refid="Apply-Orbit-File(1)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
     

##### 2.4. Write

In [37]:
operator = 'Write'

params_list_master = []

for index, slave_split_node in enumerate(slave_split_nodes):
    parameters = dict()
    
    output_name_master = os.path.join(temp_folder, 'temp_master_%s' % index)
    
    node_id = 'Write(%s)' % index
    
    source_node = slave_split_node

    for param in get_snap_parameters(operator):

        if param.getName() == 'file':

            param_value = output_name_master

        elif param.getName() == 'formatName':

            param_value = 'BEAM-DIMAP'

        else:

            param_value = param.getDefaultValue()


        print (param.getName(), param_value)

        parameters[param.getName()] = param_value
        

        print(node_id)

        print(source_node)
        
    params_list_master.append(parameters)

('file', 'temp/temp_master_0')
Write(0)
TOPSAR-Split(0)
('formatName', 'BEAM-DIMAP')
Write(0)
TOPSAR-Split(0)
('deleteOutputOnFailure', 'true')
Write(0)
TOPSAR-Split(0)
('writeEntireTileRows', 'true')
Write(0)
TOPSAR-Split(0)
('clearCacheAfterRowWrite', 'false')
Write(0)
TOPSAR-Split(0)
('file', 'temp/temp_master_1')
Write(1)
TOPSAR-Split(1)
('formatName', 'BEAM-DIMAP')
Write(1)
TOPSAR-Split(1)
('deleteOutputOnFailure', 'true')
Write(1)
TOPSAR-Split(1)
('writeEntireTileRows', 'true')
Write(1)
TOPSAR-Split(1)
('clearCacheAfterRowWrite', 'false')
Write(1)
TOPSAR-Split(1)
('file', 'temp/temp_master_2')
Write(2)
TOPSAR-Split(2)
('formatName', 'BEAM-DIMAP')
Write(2)
TOPSAR-Split(2)
('deleteOutputOnFailure', 'true')
Write(2)
TOPSAR-Split(2)
('writeEntireTileRows', 'true')
Write(2)
TOPSAR-Split(2)
('clearCacheAfterRowWrite', 'false')
Write(2)
TOPSAR-Split(2)


In [38]:
slave_tr_nodes = []

for index, slave_split_node in enumerate(slave_split_nodes):
    
    node_id = 'Write(%s)' % index
    
    source_node = slave_split_node
    
    mygraph.add_node(node_id, operator, params_list_master[index], source_node)
    
    slave_tr_nodes.append(node_id)
      
    print(node_id)
    
    print(source_node)

Write(0)
TOPSAR-Split(0)
Write(1)
TOPSAR-Split(1)
Write(2)
TOPSAR-Split(2)


In [39]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read(1)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/mall/data/S1A_IW_SLC__1SDV_20170101T092231_20170101T092258_014636_017CCA_B6F3/S1A_IW_SLC__1SDV_20170101T092231_20170101T092258_014636_017CCA_B6F3.SAFE/manifest.safe</file>
    </parameters>
  </node>
  <node id="Apply-Orbit-File(1)">
    <operator>Apply-Orbit-File</operator>
    <sources>
      <sourceProduct refid="Read(1)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <polyDegree>3</polyDegree>
      <orbitType>Sentinel Precise (Auto Download)</orbitType>
      <continueOnFail>false</continueOnFail>
    </parameters>
  </node>
  <node id="TOPSAR-Split(0)">
    <operator>TOPSAR-Split</operator>
    <sources>
      <sourceProduct refid="Apply-Orbit-File(1)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
     

##### 1.10. Run

In [40]:
mygraph.run()

1937
Executing processing graph
....10%....20%....30%....40%....50%....60%....70%....80%....90% done.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
SEVERE: org.esa.s2tbx.dataio.gdal.activator.GDALDistributionInstaller: The environment variable LD_LIBRARY_PATH is not set. It must contain the current folder '.'.
INFO: org.hsqldb.persist.Logger: dataFileCache open start
INFO: org.esa.snap.engine_utilities.download.downloadablecontent.DownloadableContentImpl: http retrieving http://step.esa.int/auxdata/orbits/Sentinel-1/POEORB/S1A/2016/12/S1A_OPER_AUX_POEORB_OPOD_20170121T121401_V20161231T225943_20170102T005942.EOF.zip



#### 3. Processing

In [41]:
mygraph = GraphProcessor()

##### 3.1. Read slave and master products

In [42]:
k = 0

read_nodes_list = []

for master, slave in zip(params_list_master, params_list_slave):
    input_products = [master['file'] + '.dim', slave['file'] + '.dim']

    operator = 'Read'

    read_nodes = []

    for index, identifier in enumerate(input_products):

        parameters = dict()

        for param in get_snap_parameters(operator):

            if param.getName() == 'file':
                parameters[param.getName()] = identifier
            else:
                parameters[param.getName()] = param.getDefaultValue()
        node_id = 'Read_%s' % (index + k * 2)

        read_nodes.append(node_id)
        
        print ('-----')
        print (node_id)
        print(parameters)

        mygraph.add_node(node_id, 'Read', parameters, '')
        
    read_nodes_list.append(read_nodes)
        
    k = k + 1


-----
Read_0
{'formatName': None, 'file': 'temp/temp_master_0.dim'}
-----
Read_1
{'formatName': None, 'file': 'temp/temp_slave_0.dim'}
-----
Read_2
{'formatName': None, 'file': 'temp/temp_master_1.dim'}
-----
Read_3
{'formatName': None, 'file': 'temp/temp_slave_1.dim'}
-----
Read_4
{'formatName': None, 'file': 'temp/temp_master_2.dim'}
-----
Read_5
{'formatName': None, 'file': 'temp/temp_slave_2.dim'}


In [43]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read_0">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_master_0.dim</file>
    </parameters>
  </node>
  <node id="Read_1">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_slave_0.dim</file>
    </parameters>
  </node>
  <node id="Read_2">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_master_1.dim</file>
    </parameters>
  </node>
  <node id="Read_3">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_slave_1.dim</file>
    </parameters>
  </node>
  <node id="Read_4">
    <operator>Read</operator>
    <sources/>
    <parameters class="

##### 3.2. Back-Geocoding for all slave and master products

In [44]:
operator = 'Back-Geocoding'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()

('demName', 'SRTM 3Sec')
('demResamplingMethod', 'BICUBIC_INTERPOLATION')
('externalDEMFile', None)
('externalDEMNoDataValue', '0')
('resamplingType', 'BISINC_5_POINT_INTERPOLATION')
('maskOutAreaWithoutElevation', 'true')
('outputRangeAzimuthOffset', 'false')
('outputDerampDemodPhase', 'false')
('disableReramp', 'false')


In [45]:
index = 0
cor_nodes_list = []

for node in read_nodes_list:
    cor_nodes = []

    node_id = 'Back-Geocoding_%s' % index

    source_nodes = read_nodes

    mygraph.add_node(node_id, operator, parameters, node)

    cor_nodes.append(node_id)

    print(node_id)

    print(node)
    
    index = index + 1 
    
    cor_nodes_list.append(cor_nodes)

Back-Geocoding_0
['Read_0', 'Read_1']
Back-Geocoding_1
['Read_2', 'Read_3']
Back-Geocoding_2
['Read_4', 'Read_5']


In [46]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read_0">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_master_0.dim</file>
    </parameters>
  </node>
  <node id="Read_1">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_slave_0.dim</file>
    </parameters>
  </node>
  <node id="Read_2">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_master_1.dim</file>
    </parameters>
  </node>
  <node id="Read_3">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_slave_1.dim</file>
    </parameters>
  </node>
  <node id="Read_4">
    <operator>Read</operator>
    <sources/>
    <parameters class="

##### 3.3. Coherence

In [47]:
operator = 'Coherence'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()

('cohWinAz', '10')
('cohWinRg', '10')
('subtractFlatEarthPhase', 'false')
('srpPolynomialDegree', '5')
('srpNumberPoints', '501')
('orbitDegree', '3')
('squarePixel', 'true')
('subtractTopographicPhase', 'false')
('demName', 'SRTM 3Sec')
('externalDEMFile', None)
('externalDEMNoDataValue', '0')
('externalDEMApplyEGM', 'true')
('tileExtensionPercent', '100')


In [48]:
index = 0
fil_nodes_list = []

for node in cor_nodes_list:
    node_id = 'Coherence_%s' % index
    
    fil_nodes = []

    source_nodes = cor_nodes

    mygraph.add_node(node_id, operator, parameters, node)

    fil_nodes.append(node_id)

    print(node_id)

    print(node)
    
    fil_nodes_list.append(fil_nodes)
    
    index = index + 1

Coherence_0
['Back-Geocoding_0']
Coherence_1
['Back-Geocoding_1']
Coherence_2
['Back-Geocoding_2']


In [49]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read_0">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_master_0.dim</file>
    </parameters>
  </node>
  <node id="Read_1">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_slave_0.dim</file>
    </parameters>
  </node>
  <node id="Read_2">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_master_1.dim</file>
    </parameters>
  </node>
  <node id="Read_3">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_slave_1.dim</file>
    </parameters>
  </node>
  <node id="Read_4">
    <operator>Read</operator>
    <sources/>
    <parameters class="

##### 3.4. TOPSAR-Deburst

In [50]:
operator = 'TOPSAR-Deburst'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    parameters[param.getName()] = param.getDefaultValue()

('selectedPolarisations', None)


In [51]:
index = 0
deb_nodes_list = []

for node in fil_nodes_list:
    
    node_id = 'TOPSAR-Deburst_%s' % index

    deb_nodes = []

    source_node = deb_nodes

    mygraph.add_node(node_id, operator, parameters, node)

    deb_nodes.append(node_id)

    print(node_id)

    print(node)
        
    deb_nodes_list.append(deb_nodes)
    
    index = index + 1

TOPSAR-Deburst_0
['Coherence_0']
TOPSAR-Deburst_1
['Coherence_1']
TOPSAR-Deburst_2
['Coherence_2']


In [52]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read_0">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_master_0.dim</file>
    </parameters>
  </node>
  <node id="Read_1">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_slave_0.dim</file>
    </parameters>
  </node>
  <node id="Read_2">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_master_1.dim</file>
    </parameters>
  </node>
  <node id="Read_3">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_slave_1.dim</file>
    </parameters>
  </node>
  <node id="Read_4">
    <operator>Read</operator>
    <sources/>
    <parameters class="

In [53]:
deb_nodes_list

[['TOPSAR-Deburst_0'], ['TOPSAR-Deburst_1'], ['TOPSAR-Deburst_2']]

##### 3.5. TOPSAR-Merge

In [54]:
operator = 'TOPSAR-Merge'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    parameters[param.getName()] = param.getDefaultValue()

('selectedPolarisations', None)


In [55]:
merge_nodes = []

node_id = 'TOPSAR-Merge(%s)' % str(0)

source_nodes = []

for index, deb_node in enumerate(deb_nodes_list):
    
    source_nodes.append(deb_node[0])
    
mygraph.add_node(node_id, operator, parameters, source_nodes)

merge_node = node_id

print(node_id)
    
print(source_nodes)

TOPSAR-Merge(0)
['TOPSAR-Deburst_0', 'TOPSAR-Deburst_1', 'TOPSAR-Deburst_2']


mygraph.view_graph()

##### 3.6. Multilook

In [56]:
operator = 'Multilook'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    if param.getName() == 'nRgLooks':
        parameters[param.getName()] = '4'
    elif param.getName() == 'outputIntensity':
        parameters[param.getName()] = 'false'
    else:
        parameters[param.getName()] = param.getDefaultValue()

('sourceBandNames', None)
('nRgLooks', '1')
('nAzLooks', '1')
('outputIntensity', 'false')
('grSquarePixel', 'true')


In [57]:
index = 0
    
node_id = 'Multilook_%s' % index
    
source_node = merge_node
    
mygraph.add_node(node_id, operator, parameters, source_node)
    
ml_node = node_id
    
print(node_id)
    
print(source_node)

Multilook_0
TOPSAR-Merge(0)


In [58]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read_0">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_master_0.dim</file>
    </parameters>
  </node>
  <node id="Read_1">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_slave_0.dim</file>
    </parameters>
  </node>
  <node id="Read_2">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_master_1.dim</file>
    </parameters>
  </node>
  <node id="Read_3">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_slave_1.dim</file>
    </parameters>
  </node>
  <node id="Read_4">
    <operator>Read</operator>
    <sources/>
    <parameters class="

##### 3.7. Terrain-Correction

In [59]:
operator = 'Terrain-Correction'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()


('sourceBandNames', None)
('demName', 'SRTM 3Sec')
('externalDEMFile', None)
('externalDEMNoDataValue', '0')
('externalDEMApplyEGM', 'true')
('demResamplingMethod', 'BILINEAR_INTERPOLATION')
('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
('pixelSpacingInMeter', '0')
('pixelSpacingInDegree', '0')
('mapProjection', 'WGS84(DD)')
('alignToStandardGrid', 'false')
('standardGridOriginX', '0')
('standardGridOriginY', '0')
('nodataValueAtSea', 'true')
('saveDEM', 'false')
('saveLatLon', 'false')
('saveIncidenceAngleFromEllipsoid', 'false')
('saveLocalIncidenceAngle', 'false')
('saveProjectedLocalIncidenceAngle', 'false')
('saveSelectedSourceBand', 'true')
('outputComplex', 'false')
('applyRadiometricNormalization', 'false')
('saveSigmaNought', 'false')
('saveGammaNought', 'false')
('saveBetaNought', 'false')
('incidenceAngleForSigma0', 'Use projected local incidence angle from DEM')
('incidenceAngleForGamma0', 'Use projected local incidence angle from DEM')
('auxFile', 'Latest Auxiliary Fil

In [60]:
index = 0

node_id = 'Terrain-Correction'

source_nodes = ml_node

mygraph.add_node(node_id, operator, parameters, source_nodes)

terrain_node = node_id

print(node_id)
    
print(source_nodes)


Terrain-Correction
Multilook_0


In [61]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read_0">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_master_0.dim</file>
    </parameters>
  </node>
  <node id="Read_1">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_slave_0.dim</file>
    </parameters>
  </node>
  <node id="Read_2">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_master_1.dim</file>
    </parameters>
  </node>
  <node id="Read_3">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_slave_1.dim</file>
    </parameters>
  </node>
  <node id="Read_4">
    <operator>Read</operator>
    <sources/>
    <parameters class="

##### 3.8. Write

In [62]:
operator = 'Write'

output_name = os.path.join(temp_folder, 'temp_merge')

parameters = dict()

for param in get_snap_parameters(operator):
    
    if param.getName() == 'file':
        
        param_value = output_name
             
    elif param.getName() == 'formatName':
                
        param_value = 'BEAM-DIMAP'
        
    else:
    
        param_value = param.getDefaultValue()
    
    
    print (param.getName(), param_value)
    
    parameters[param.getName()] = param_value

('file', 'temp/temp_merge')
('formatName', 'BEAM-DIMAP')
('deleteOutputOnFailure', 'true')
('writeEntireTileRows', 'true')
('clearCacheAfterRowWrite', 'false')


In [63]:
mygraph.add_node(operator, 
             operator, 
             parameters,
             terrain_node)

In [64]:
mygraph.view_graph() 

<graph>
  <version>1.0</version>
  <node id="Read_0">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_master_0.dim</file>
    </parameters>
  </node>
  <node id="Read_1">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_slave_0.dim</file>
    </parameters>
  </node>
  <node id="Read_2">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_master_1.dim</file>
    </parameters>
  </node>
  <node id="Read_3">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>temp/temp_slave_1.dim</file>
    </parameters>
  </node>
  <node id="Read_4">
    <operator>Read</operator>
    <sources/>
    <parameters class="

In [ ]:
mygraph.run()

2209


#### Write result in GeoTIFF

In [ ]:
output_name = 'coherence'

reader = ProductIO.getProductReader("BEAM-DIMAP")

b0ch = reader.readProductNodes(output_name + '.dim', None)

In [ ]:
temp_tif_name = os.path.join(temp_folder, output_name + '.tif')

ProductIO.wrfiteProduct(b0ch, 
                       temp_tif_name,
                       'GeoTIFF-BigTiff')

In [ ]:
crop_image(temp_tif_name,regionOfInterest['value'], output_path)

##### 4.1. Read slave and master products

In [ ]:
mygraph = GraphProcessor()

In [ ]:
master_value = master['value']

if master_value== 'True':

    k = 0

    read_nodes_list = []

    for master in params_list_master:
        input_products = [master['file'] + '.dim']

        operator = 'Read'

        read_nodes = []

        for index, identifier in enumerate(input_products):

            parameters = dict()

            for param in get_snap_parameters(operator):

                if param.getName() == 'file':
                    parameters[param.getName()] = identifier
                else:
                    parameters[param.getName()] = param.getDefaultValue()
            node_id = 'Read_%s' % (index + k)

            read_nodes.append(node_id)

            print ('-----')
            print (node_id)
            print(parameters)

            mygraph.add_node(node_id, 'Read', parameters, '')

        read_nodes_list.append(read_nodes)

        k = k + 1


In [ ]:
mygraph.view_graph()

4.2. ThermalNoiseRemoval

In [ ]:
operator = 'ThermalNoiseRemoval'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()

In [ ]:
master_value = master['value']

if master_value== 'True':
    
    index = 0
    cor_nodes_list = []

    for node in read_nodes_list:
        cor_nodes = []

        node_id = 'ThermalNoiseRemoval_%s' % index

        source_nodes = read_nodes

        mygraph.add_node(node_id, operator, parameters, node)

        cor_nodes.append(node_id)

        print(node_id)

        print(node)

        index = index + 1 

        cor_nodes_list.append(cor_nodes)

In [ ]:
mygraph.view_graph()

##### 4.3. Calibration

In [ ]:
operator = 'Calibration'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    if param.getName() == 'outputSigmaBand':
        parameters[param.getName()] = 'true'
    elif param.getName() == 'outputBetaBand':
        parameters[param.getName()] = 'false'      
    else:
        parameters[param.getName()] = param.getDefaultValue()

In [ ]:
master_value = master['value']

if master_value== 'True':
    
    index = 0
    fil_nodes_list = []

    for node in cor_nodes_list:
        node_id = 'Calibration_%s' % index

        fil_nodes = []

        source_nodes = cor_nodes

        mygraph.add_node(node_id, operator, parameters, node)

        fil_nodes.append(node_id)

        print(node_id)

        print(node)

        fil_nodes_list.append(fil_nodes)

        index = index + 1

In [ ]:
mygraph.view_graph()

##### 4.4. TOPSAR-Deburst

In [ ]:
operator = 'TOPSAR-Deburst'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    parameters[param.getName()] = param.getDefaultValue()

In [ ]:
master_value = master['value']

if master_value== 'True':
    
    index = 0
    deb_nodes_list = []

    for node in fil_nodes_list:

        node_id = 'TOPSAR-Deburst_%s' % index

        deb_nodes = []

        source_node = deb_nodes

        mygraph.add_node(node_id, operator, parameters, node)

        deb_nodes.append(node_id)

        print(node_id)

        print(node)

        deb_nodes_list.append(deb_nodes)

        index = index + 1

In [ ]:
mygraph.view_graph()

##### 4.5. TOPSAR-Merge

In [ ]:
operator = 'TOPSAR-Merge'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    parameters[param.getName()] = param.getDefaultValue()

In [ ]:
master_value = master['value']

if master_value== 'True':
    
    merge_nodes = []

    node_id = 'TOPSAR-Merge(%s)' % str(0)

    source_nodes = []

    for index, deb_node in enumerate(deb_nodes_list):

        source_nodes.append(deb_node[0])

    mygraph.add_node(node_id, operator, parameters, source_nodes)

    merge_node = node_id

    print(node_id)

    print(source_nodes)

In [ ]:
mygraph.view_graph()

##### 4.6. Multilook

In [ ]:
operator = 'Multilook'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    if param.getName() == 'nRgLooks':
        parameters[param.getName()] = '4'
    elif param.getName() == 'outputIntensity':
        parameters[param.getName()] = 'false'
    else:
        parameters[param.getName()] = param.getDefaultValue()

In [ ]:
master_value = master['value']

if master_value== 'True':
    
    index = 0

    node_id = 'Multilook_%s' % index

    source_node = merge_node

    mygraph.add_node(node_id, operator, parameters, source_node)

    ml_node = node_id

    print(node_id)

    print(source_node)

In [ ]:
mygraph.view_graph()

##### 4.7. Terrain-Correction

In [ ]:
operator = 'Terrain-Correction'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()


In [ ]:
master_value = master['value']

if master_value== 'True':

    index = 0

    node_id = 'Terrain-Correction'

    source_nodes = ml_node

    mygraph.add_node(node_id, operator, parameters, source_nodes)

    terrain_node = node_id

    print(node_id)

    print(source_nodes)


In [ ]:
mygraph.view_graph()

##### 4.8. Write

In [ ]:
master_value = master['value']

if master_value== 'True':
    
    operator = 'Write'

    output_name = os.path.join(temp_folder, 'temp_master_intensity')

    parameters = dict()

    for param in get_snap_parameters(operator):

        if param.getName() == 'file':

            param_value = output_name

        elif param.getName() == 'formatName':

            param_value = 'BEAM-DIMAP'

        else:

            param_value = param.getDefaultValue()


        print (param.getName(), param_value)

        parameters[param.getName()] = param_value

In [ ]:
master_value = master['value']

if master_value== 'True':
    
    mygraph.add_node(operator, 
                 operator, 
                 parameters,
                 terrain_node)

In [ ]:
mygraph.view_graph() 

In [ ]:
mygraph.run() 

#### Write result in GeoTIFF

In [ ]:
output_name = 'intensity_master'

reader = ProductIO.getProductReader("BEAM-DIMAP")

b0ch = reader.readProductNodes(output_name + '.dim', None)

In [ ]:
temp_tif_name = os.path.join(temp_folder, output_name + '.tif')

ProductIO.writeProduct(b0ch, 
                       temp_tif_name,
                       'GeoTIFF-BigTiff')

In [ ]:
crop_image(temp_tif_name,regionOfInterest['value'], output_path)

##### 5.1. Read slave products

In [ ]:
mygraph = GraphProcessor()

In [ ]:
slave_value = slave['value']

if slave_value== 'True':

    k = 0

    read_nodes_list = []

    for slave in params_list_slave:
        input_products = [master['file'] + '.dim']

        operator = 'Read'

        read_nodes = []

        for index, identifier in enumerate(input_products):

            parameters = dict()

            for param in get_snap_parameters(operator):

                if param.getName() == 'file':
                    parameters[param.getName()] = identifier
                else:
                    parameters[param.getName()] = param.getDefaultValue()
            node_id = 'Read_%s' % (index + k)

            read_nodes.append(node_id)

            print ('-----')
            print (node_id)
            print(parameters)

            mygraph.add_node(node_id, 'Read', parameters, '')

        read_nodes_list.append(read_nodes)

        k = k + 1


In [ ]:
mygraph.view_graph()

5.2. ThermalNoiseRemoval

In [ ]:
operator = 'ThermalNoiseRemoval'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()

In [ ]:
slave_value = slave['value']

if slave_value== 'True':
    
    index = 0
    cor_nodes_list = []

    for node in read_nodes_list:
        cor_nodes = []

        node_id = 'ThermalNoiseRemoval_%s' % index

        source_nodes = read_nodes

        mygraph.add_node(node_id, operator, parameters, node)

        cor_nodes.append(node_id)

        print(node_id)

        print(node)

        index = index + 1 

        cor_nodes_list.append(cor_nodes)

In [ ]:
mygraph.view_graph()

##### 5.3. Calibration

In [ ]:
operator = 'Calibration'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    if param.getName() == 'outputSigmaBand':
        parameters[param.getName()] = 'true'
    elif param.getName() == 'outputBetaBand':
        parameters[param.getName()] = 'false'      
    else:
        parameters[param.getName()] = param.getDefaultValue()

In [ ]:
slave_value = slave['value']

if slave_value== 'True':
    
    index = 0
    fil_nodes_list = []

    for node in cor_nodes_list:
        node_id = 'Calibration_%s' % index

        fil_nodes = []

        source_nodes = cor_nodes

        mygraph.add_node(node_id, operator, parameters, node)

        fil_nodes.append(node_id)

        print(node_id)

        print(node)

        fil_nodes_list.append(fil_nodes)

        index = index + 1

In [ ]:
mygraph.view_graph()

##### 5.4. TOPSAR-Deburst

In [ ]:
operator = 'TOPSAR-Deburst'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    parameters[param.getName()] = param.getDefaultValue()

In [ ]:
slave_value = slave['value']

if slave_value== 'True':
    
    index = 0
    deb_nodes_list = []

    for node in fil_nodes_list:

        node_id = 'TOPSAR-Deburst_%s' % index

        deb_nodes = []

        source_node = deb_nodes

        mygraph.add_node(node_id, operator, parameters, node)

        deb_nodes.append(node_id)

        print(node_id)

        print(node)

        deb_nodes_list.append(deb_nodes)

        index = index + 1

In [ ]:
mygraph.view_graph()

##### 5.5. TOPSAR-Merge

In [ ]:
operator = 'TOPSAR-Merge'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    parameters[param.getName()] = param.getDefaultValue()

In [ ]:
slave_value = slave['value']

if slave_value== 'True':
    
    merge_nodes = []

    node_id = 'TOPSAR-Merge(%s)' % str(0)

    source_nodes = []

    for index, deb_node in enumerate(deb_nodes_list):

        source_nodes.append(deb_node[0])

    mygraph.add_node(node_id, operator, parameters, source_nodes)

    merge_node = node_id

    print(node_id)

    print(source_nodes)

In [ ]:
mygraph.view_graph()

##### 5.6. Multilook

In [ ]:
operator = 'Multilook'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    if param.getName() == 'nRgLooks':
        parameters[param.getName()] = '4'
    elif param.getName() == 'outputIntensity':
        parameters[param.getName()] = 'false'
    else:
        parameters[param.getName()] = param.getDefaultValue()

In [ ]:
slave_value = slave['value']

if slave_value== 'True':
    
    index = 0

    node_id = 'Multilook_%s' % index

    source_node = merge_node

    mygraph.add_node(node_id, operator, parameters, source_node)

    ml_node = node_id

    print(node_id)

    print(source_node)

In [ ]:
mygraph.view_graph()

##### 5.7. Terrain-Correction

In [ ]:
operator = 'Terrain-Correction'

parameters = dict()

for param in get_snap_parameters(operator):
    
    print(param.getName(), param.getDefaultValue())
    
    parameters[param.getName()] = param.getDefaultValue()


In [ ]:
slave_value = slave['value']

if slave_value== 'True':

    index = 0

    node_id = 'Terrain-Correction'

    source_nodes = ml_node

    mygraph.add_node(node_id, operator, parameters, source_nodes)

    terrain_node = node_id

    print(node_id)

    print(source_nodes)


In [ ]:
mygraph.view_graph()

##### 5.8. Write

In [ ]:
slave_value = slave['value']

if slave_value== 'True':
    
    operator = 'Write'

    output_name = os.path.join(temp_folder, 'temp_slave_intensity')

    parameters = dict()

    for param in get_snap_parameters(operator):

        if param.getName() == 'file':

            param_value = output_name

        elif param.getName() == 'formatName':

            param_value = 'BEAM-DIMAP'

        else:

            param_value = param.getDefaultValue()


        print (param.getName(), param_value)

        parameters[param.getName()] = param_value

In [ ]:
slave_value = slave['value']

if slave_value== 'True':
    
    mygraph.add_node(operator, 
                 operator, 
                 parameters,
                 terrain_node)

In [ ]:
mygraph.view_graph() 

In [ ]:
mygraph.run()

#### Write result in GeoTIFF

In [ ]:
output_name = 'intensity_slave'

reader = ProductIO.getProductReader("BEAM-DIMAP")

b0ch = reader.readProductNodes(output_name + '.dim', None)

In [ ]:
temp_tif_name = os.path.join(temp_folder, output_name + '.tif')

ProductIO.writeProduct(b0ch, 
                       temp_tif_name,
                       'GeoTIFF-BigTiff')

In [ ]:
crop_image(temp_tif_name,regionOfInterest['value'], output_path)

#### Clean-up

In [ ]:
shutil.rmtree(temp_folder)

shutil.rmtree(output_name + '.data')
os.remove(output_name + '.dim')

### <a name="License"> License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.